# Parsing a data2dome feed to create Uniview Modules with fulldome images

This notebook shows how to connect to a [data2dome](https://docs.google.com/document/d/1USG1clnxSyGf9lsDe-alb6nZzxn_xFzxbjCnSVPzlWU/edit#) JSON feed.

In this example we query the ESO images feed and search for recent images in fulldome format. We then construct an html table with thumbnail images so that we can select which ones we want. Finally a Uniview module is created to hold the fisheye image.

Mark SubbaRao, April 25th, 2017

In [458]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import date, timedelta
import requests
from PIL import Image as Im
import os

# D2D Feed
ESOimages = 'http://www.eso.org/public/images/d2d/'
howRecent = 200 # in days
pd.set_option('max_colwidth',5000) # so we will return the full description without truncation
numReturn=20 # maximum number of results to return
desiredResolution = 4096

## Parse the Feed

This code block downloads the pagenated json feed and parses it selecting only the images in fulldome format until numReturn objects are found.

In [272]:
page = ESOimages+'?after='+(date.today()-timedelta(days=howRecent)).strftime('%Y%m%d')+'0000' #the 0000 is the time component of the date
nImg = 0
nPage=0
feedImages=[]
while (page!='' and nImg < numReturn):
    print(page)
    df=pd.read_json(page)
    # The Next parameter may or may not exist
    try:
        page=df.Next[0]
    except:
        page=''
    nPage=nPage+1
    for feedImage in df.Collections:
        if (feedImage['Assets'][0]['Resources'][0]['ProjectionType']=='Fulldome'and nImg < numReturn):
            feedImages.append(feedImage)
            nImg=nImg+1
    print(str(nPage) + ' pages parsed ' + str(nImg) + ' fulldome images found')

http://www.eso.org/public/images/d2d/?after=201610060000
1 pages parsed 1 fisheye images found
http://www.eso.org/public/images/d2d/?after=201610060000&page=2
2 pages parsed 6 fisheye images found
http://www.eso.org/public/images/d2d/?after=201610060000&page=3
3 pages parsed 11 fisheye images found
http://www.eso.org/public/images/d2d/?after=201610060000&page=4
4 pages parsed 13 fisheye images found


## Create table with thumbnail images for image selection

In [285]:
titleList=[]
descriptionList=[]
thumbnailList=[]
pubdateList=[]
for pic in feedImages:
    titleList.append(pic['Title'])
    descriptionList.append(pic['Description'].replace("\r\n",''))
    pubdateList.append(pic['PublicationDate'])
    for resource in pic['Assets'][0]['Resources']:
        if (resource['ResourceType']=='Thumbnail'):
            thumbnailList.append('<img src="'+resource['URL']+'"/>')            

df3 = pd.DataFrame({'title':titleList,'description':descriptionList,'thumbnail':thumbnailList,'pubdate':pubdateList},columns=['title','thumbnail','description','pubdate'])

In [286]:
HTML(df3.to_html(escape=False))

,title,thumbnail,description,pubdate
0,Fulldome selfie,,"Beneath the grand arc of the Milky Way, ESO Photo Ambassador Petr Horálek is just a small speck in an enormous Universe. Horálek is standing amongst the antennas of the Atacama Large Millimeter/submillimeter Array (ALMA), which studies some of the coldest objects in the Universe.",2017-02-24T13:58:20Z
1,Paranal bow — fisheye,,The four huge Unit Telescopes (UTs) at ESO's Paranal Observatory in Chile are among the most advanced in the world. The perfect backdrop is provided by a view on the galaxy in which we live — the Milky Way.LinksExtended to 360 x 180 degrees (with black) version of this imageEquirectangular projected version of this image,2017-01-18T14:10:33Z
2,Milky Way between UTs — fisheye,,"The Milky Way tucks in nicely between the two Unit Telescope (UT) behemoths that make up part of ESO's Very Large Telescope in Chile. As can be seen from the brilliant Milky Way, the skies here are crystal-clear, a result of the lack of light pollution and high altitude.LinksExtended to 360 x 180 degrees (with black) version of this image",2017-01-18T10:50:51Z
3,Dawn at La Silla — fisheye,,"The rising Sun brings an end to observations at ESO's La Silla Observatory in the Atacama desert of Chile. La Silla was ESO's first observatory in Chile, the telescopes there taking their first light from the Universe in the late 1960s.LinksExtended to 360 x 180 degrees (with black) version of this image",2017-01-18T10:45:26Z
4,Razor-sharp Paranal,,"New webcams installed at ESO's facilities in Chile provide unparalleled levels of detail of the world's most productive astronomical observatories. Here, the Very Large Telescope is imaged in 4K definition, beneath the cloudless Atacama sky.Such images could be used in the ESO Supernova Planetarium & Visitor Centre in Garching, Germany.",2016-11-30T14:24:39.419215Z
5,That much closer,,"With light pollution and cloud cover rarely issues at ESO's Paranal Observatory, the Universe can seem closer than ever. Here, one of ESO's Photo Ambassadors, Petr Horálek, gets a better view on the Very Large Telescope, nearer the heavens.Images like this were gathered on the Fulldome Expedition, an endeavour to produce visuals for the ESO Supernova Planetarium & Visitor Centre in Garching, Munich.",2016-11-28T12:20:48.677643Z
6,ALMA gegenschein — fisheye,,"On the Chajnantor plateau in Chile, home of the Atacama Large Millimetre/submillimetre Array (ALMA), the sky is so dark that the famous and extremely difficult to observe gegenschein (or ""countershine"") is sometimes visible.This is a faint brightening of the night sky in the region of the ecliptic directly opposite the Sun, caused by reflection of sunlight by interplanetary dust in the Solar System.LinksExtended to 360 x 180 degrees (with black) version of this image",2016-11-24T13:48:26Z
7,Nightfall Residencia — fisheye,,"An oasis in an otherwise barren desert, the Paranal Residencia provides accommodation for the scientists, engineers and technicians who work at ESO's Very Large Telescope in Chile. It's not everywhere in the world where you can get a view like this from your bedroom window.LinksExtended to 360 x 180 degrees (with black) version of this image",2016-11-23T13:22:29.707760Z
8,Four units,,"The four large Unit Telescopes (UTs) make up the bulk of ESO's Very Large Telescope, located at Paranal Observatory in the Chilean desert. This fisheye photograph shows the UTs in action.Such an image could be shown in planetariums worldwide, such as in the ESO Supernova Planetarium & Visitor Centre in Munich, Germany.",2016-11-16T14:08:27.352267Z
9,Milky Way slice,,"The wide arch of the Milky Way cuts through the centre of this fisheye photograph of ESO's Paranal Observatory, which is home to the Very Large Telescope. One of the facility's Auxiliary Telescopes is wide-eyed and taking in light from the Universe's phenomena.Such an image could be shown in planetariums worldwide, such as in the ESO Supernova Pl

## Selecting an image and creating a Uniview module

The next code block contains a method to build a Uniview module given a name and the number of the image, it:

1. Creates a Directory in Uniview Custom Modules
1. Downloads the image, resizes it if necessary and saves it in module directory
1. Creates a .mod file for this module
1. Creates a module.definition file, which holds the title, credit, rights and publication date from the D2D feed

In [456]:
UniviewModulePath = r'C:\Users\msubbarao\SCISS\Uniview Theater 2.0\Custom Modules'
def makeUVmodule(num,moduleName):
    imURL=''
    imSize=0
    imDict = feedImages[num]
    
    # FIRST Create a Directory in Uniview Custom Modules
    UniviewModuleDir=os.path.join(UniviewModulePath,moduleName)
    if not os.path.exists(UniviewModuleDir):
        os.makedirs(UniviewModuleDir)
    os.path.exists(UniviewModuleDir)    
    
    # SECOND Download image, resize if necessary and save in module directory
    for resource in imDict['Assets'][0]['Resources']:
        if ((resource['Dimensions'][0]>=imSize and imSize < desiredResolution) or (resource['Dimensions'][0]<=imSize and imSize > desiredResolution and resource['Dimensions'][0] > desiredResolution)):
            imSize = resource['Dimensions'][0]
            imURL = resource['URL']
    print('downloading '+imURL)    
    im = Im.open(requests.get(imURL, stream=True).raw)
    if (imSize > desiredResolution):
        im = im.resize((desiredResolution,desiredResolution))
        print('resizing image from '+str(imSize)+' to '+str(desiredResolution))
    imageName=moduleName+'.jpg'
    im.save(os.path.join(UniviewModuleDir,imageName))
    
    # THIRD create a .mod file for this module
    modFileText='''
    filepath +:.:./modules/'''+moduleName+'''
    
    2dobject ''' + moduleName + ''' sgFisheyeScreenRenderObject
    {
    prop.Enabled false
    prop.Depth 200
    prop.fileName ./modules/'''+moduleName+'/'+imageName+'''
    prop.alpha 1.0
    prop.tilt -45
    }'''
    print('writing .mod file')
    modFileName=moduleName+'.mod'
    writeFile = open(os.path.join(UniviewModuleDir,modFileName),'w')
    writeFile.write(modFileText)
    writeFile.close()
    
    # FORTH create a module.definition file
    definitionText='''
    <uniview>
    <name>'''+moduleName+'''</name>
    <description>'''+imDict["Title"]+'''</description>
    <creator>'''+imDict["Credit"]+'''</creator>
    <legalinfo>'''+imDict["Rights"]+'''</legalinfo>
    <version></version>
    <creationdate>'''+imDict["PublicationDate"]+'''</creationdate>
    </uniview>
    '''
    print('writing module.definition file')
    writeFile = open(os.path.join(UniviewModuleDir,'module.definition'),'w')
    writeFile.write(definitionText)
    writeFile.close()
    print('Done :)')

Here is an example of how to call the makeUVmodule method

In [457]:
makeUVmodule(4,'VLT_4')

downloading https://cdn.eso.org/images/large/preset-99.jpg
resizing image from 4373 to 4096
writing .mod file
writing module.definition file
Done :)
